In [1]:
import os

In [2]:
os.chdir("./../.")

In [3]:
import dotenv
from pymongo import MongoClient

In [4]:
from app import utils

In [ ]:
dotenv.load_dotenv()

In [6]:
mongodb_url = os.environ.get("MONGODB_URL")

In [7]:
mongodb_client = MongoClient(mongodb_url)

In [8]:
db = mongodb_client['default']

### (1) Read Input Prompt

In [9]:
prompt_path = "./prompts/prompt_investment.json"

In [10]:
prompt_list = utils.read_json_file(prompt_path)

### (2) Ingest Category

In [11]:
unique_categories = list(set([p['category'] for p in prompt_list]))

In [12]:
existing_categories = set([item['category']
                           for item in db['PromptCategory'].find()])

In [13]:
new_categories = [
    {"category": category} for category in unique_categories if category not in existing_categories]

In [ ]:
db['PromptCategory'].insert_many(new_categories)

### (3) Ingest the prompt

In [15]:
userId = "public"

In [16]:
prompt_categories = list(db['PromptCategory'].find())

In [17]:
category_to_id_dict = {category['category']: category["_id"]
                       for category in prompt_categories}

In [18]:
input_prompt_docs = []

for prompt in prompt_list:
    prompt.pop("id")
    prompt['categoryId'] = category_to_id_dict[prompt['category']]
    prompt['userId'] = "public"
    prompt['share'] = True
    prompt.pop("category")
    input_prompt_docs.append(prompt)

In [ ]:
input_prompt_docs

In [ ]:
db['Prompt'].insert_many(input_prompt_docs)